In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [3]:
!ls ./MyDrive/MyDrive/omneda_sudan/

dtm_sdn_smu-bi-weekly-13-_-17122024_v02_public_hdx.xlsx
gender_sdn.csv
hdro_indicators_aggregates_sdn.csv
hdro_indicators_sdn.csv
indicators_sdn.csv
sdg_data_sdn.csv
sdg_indicatorlist_sdn.csv
WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx
WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS.xlsx


### Read in the United Nations demographic indicator data

###### WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx

In [4]:
demo_indicator = pd.read_excel('./MyDrive/MyDrive/omneda_sudan/WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx',
                               sheet_name="Estimates",
                               skiprows=15, header=[0,1])

In [5]:
demo_indicator.head()

Unnamed: 0_level_0 Unnamed: 1_level_0                   Unnamed: 2_level_0  \
               Index            Variant Region, subregion, country or area *   
0                  1          Estimates                                World   
1                  2          Estimates                                World   
2                  3          Estimates                                World   
3                  4          Estimates                                World   
4                  5          Estimates                                World   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0  \
               Notes      Location code    ISO3 Alpha-code    ISO2 Alpha-code   
0                NaN                900                NaN                NaN   
1                NaN                900                NaN                NaN   
2                NaN                900                NaN                NaN   
3                NaN                900                NaN                NaN   
4                NaN                900                NaN                NaN   

  Unnamed: 7_level_0 Unnamed: 8_level_0 Unnamed: 9_level_0  ...  \
         SDMX code**               Type        Parent code  ...   
0                1.0              World                  0  ...   
1                1.0              World                  0  ...   
2                1.0              World                  0  ...   
3                1.0              World                  0  ...   
4                1.0              World                  0  ...   

                                                                      Mortality  \
  Male Mortality before Age 60 (deaths under age 60 per 1,000 male live births)   
0                                              580.5                              
1                                            566.566                              
2                                            546.444                              
3                                            535.811                              
4                                            522.058                              

                                                                                     \
  Female Mortality before Age 60 (deaths under age 60 per 1,000 female live births)   
0                                            497.388                                  
1                                            488.435                                  
2                                             475.37                                  
3                                            467.361                                  
4                                            455.621                                  

                                                                                               \
  Mortality between Age 15 and 50, both sexes (deaths under age 50 per 1,000 alive at age 15)   
0                                            238.516                                            
1                                            229.703                                            
2                                            217.311                                            
3                                            211.257                                            
4                                            203.337                                            

                                                                                              \
  Male Mortality between Age 15 and 50 (deaths under age 50 per 1,000 males alive at age 15)   
0                                            268.734                                           
1                                            256.236                                           
2                                             238.56                                           
3                                            230.96

In [6]:
# demo_indicator.columns.values
demo_indicator.columns=['_'.join(col).strip() for col in demo_indicator.columns.values]
demo_indicator.columns=[re.findall("level_0_(.*)", col)[0]  if col.startswith('Unnamed') else col for col in demo_indicator.columns ]
demographics=demo_indicator[demo_indicator['Region, subregion, country or area *'].eq('Sudan')]

# demo_indicator.columns

### Read Aggregated Human Development Indicators for Sudan
hdro_indicators_aggregates_sdn.csv

In [9]:
hdro_indicators_agg=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/hdro_indicators_aggregates_sdn.csv')
hdro_indicators_agg.drop(0, inplace=True)
hdro_indicators_agg['year'] = hdro_indicators_agg['year'].apply(pd.to_numeric)

hdro_indicators_agg.head()

,country_code,country_name,indicator_id,indicator_name,index_id,index_name,value,year
1,SDN,Sudan,gdi,Gender Development Index (value),GDI,Gender Development Index,0.755,1990
2,SDN,Sudan,gdi,Gender Development Index (value),GDI,Gender Development Index,0.76,1991
3,SDN,Sudan,gdi,Gender Development Index (value),GDI,Gender Development Index,0.795,1992
4,SDN,Sudan,gdi,Gender Development Index (value),GDI,Gender Development Index,0.805,1993
5,SDN,Sudan,gdi,Gender Development Index (value),GDI,Gender Development Index,0.762,1994


In [10]:
# hdro_indicators.pivot(columns=['indicator_name'], values = 'value', index=['year']).reset_index().head()

### merge into demographics

In [11]:
demographics= demographics.merge(hdro_indicators_agg.pivot(columns=['indicator_name'], values = 'value', index=['year']).reset_index(),
                                 left_on='Year',
                                 right_on='year',
                                 how='left'
                                 ).drop(columns=('year'))

In [113]:
demographics.columns

Index(['Index', 'Variant', 'Region, subregion, country or area *', 'Notes',
       'Location code', 'ISO3 Alpha-code', 'ISO2 Alpha-code', 'SDMX code**',
       'Type', 'Parent code', 'Year',
       'Population_Total Population, as of 1 January (thousands)',
       'Population_Total Population, as of 1 July (thousands)',
       'Population_Male Population, as of 1 July (thousands)',
       'Population_Female Population, as of 1 July (thousands)',
       'Population_Population Density, as of 1 July (persons per square km)',
       'Population_Population Sex Ratio, as of 1 July (males per 100 females)',
       'Population_Median Age, as of 1 July (years)',
       'Population_Natural Change, Births minus Deaths (thousands)',
       'Population_Rate of Natural Change (per 1,000 population)',
       'Population_Population Change (thousands)',
       'Population_Population Growth Rate (percentage)',
       'Population_Population Annual Doubling Time (years)',
       'Fertility_Births (thousan

#### hdro_indicators_sdn.csv

In [17]:
hdro_indicators=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/hdro_indicators_sdn.csv')
hdro_indicators.drop(0, inplace=True)
hdro_indicators['year'] = hdro_indicators['year'].apply(pd.to_numeric)

hdro_indicators.head()

,country_code,country_name,indicator_id,indicator_name,index_id,index_name,value,year
1,SDN,Sudan,abr,"Adolescent Birth Rate (births per 1,000 women ...",GII,Gender Inequality Index,92.397,1990
2,SDN,Sudan,abr,"Adolescent Birth Rate (births per 1,000 women ...",GII,Gender Inequality Index,94.01,1991
3,SDN,Sudan,abr,"Adolescent Birth Rate (births per 1,000 women ...",GII,Gender Inequality Index,96.144,1992
4,SDN,Sudan,abr,"Adolescent Birth Rate (births per 1,000 women ...",GII,Gender Inequality Index,98.251,1993
5,SDN,Sudan,abr,"Adolescent Birth Rate (births per 1,000 women ...",GII,Gender Inequality Index,99.98,1994


In [18]:
demographics= demographics.merge(hdro_indicators.pivot(columns=['indicator_name'], values = 'value', index=['year']).reset_index(),
                                 left_on='Year',
                                 right_on='year',
                                 how='left'
                                 ).drop(columns=('year'))

In [12]:
# hdro_indicators.pivot(columns=['indicator_name'], values = 'value', index=['year']).reset_index().head()

In [13]:
# hdro_indicators.columns

In [14]:
# hdro_indicators.iloc[25:].head(10)

In [15]:
# hdro_indicators['index_indicator_name']=hdro_indicators['index_name'] + "_" + hdro_indicators['indicator_name']

In [16]:
# hdro_indicators.pivot(columns=  ['index_indicator_name'], values = 'value', index=['year']).reset_index().head()

In [19]:
# demographics.head().to_csv('demographics_top5.csv')

#### Indicators SDN
hdro_indicators=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/hdro_indicators_sdn.csv')
hdro_indicators.head()

In [20]:
indicators=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/indicators_sdn.csv')
indicators.drop(0, inplace=True)
indicators.drop_duplicates(subset=['Year', 'Indicator Name'], inplace=True)
# indicators.drop_duplicates(subset=['Year', 'Indicator Name', 'Value'], inplace=True)
indicators['Year'] = indicators['Year'].apply(pd.to_numeric)
indicators.head()

,Country Name,Country ISO3,Year,Indicator Name,Indicator Code,Value
1,Sudan,SDN,2022,Fertilizer consumption (kilograms per hectare ...,AG.CON.FERT.ZS,7.10201554286672
2,Sudan,SDN,2021,Fertilizer consumption (kilograms per hectare ...,AG.CON.FERT.ZS,7.10307199292779
3,Sudan,SDN,2020,Fertilizer consumption (kilograms per hectare ...,AG.CON.FERT.ZS,13.847366781552
4,Sudan,SDN,2019,Fertilizer consumption (kilograms per hectare ...,AG.CON.FERT.ZS,6.46987402618929
5,Sudan,SDN,2018,Fertilizer consumption (kilograms per hectare ...,AG.CON.FERT.ZS,7.10307199292779


In [21]:
# indicators.pivot(columns=['Indicator Name'], values = 'Value', index=['Year']).reset_index()

In [22]:
demographics= demographics.merge(indicators.pivot(columns=['Indicator Name'], values = 'Value', index=['Year']).reset_index(),
                                 left_on='Year',
                                 right_on='Year',
                                 how='left'
                                 )

#### SDG Indicator list and data

In [23]:
sdg=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/sdg_indicatorlist_sdn.csv')
sdg_data=pd.read_csv('./MyDrive/MyDrive/omneda_sudan/sdg_data_sdn.csv')
sdg.iloc[10:].head()

,indicator_id,indicator_label_en
10,AIR.1.GLAST.GPIA,Gross intake ratio to the last grade of primar...
11,AIR.1.GLAST.M,Gross intake ratio to the last grade of primar...
12,AIR.2.GPV.GLAST,Gross intake ratio to the last grade of lower ...
13,AIR.2.GPV.GLAST.F,Gross intake ratio to the last grade of lower ...
14,AIR.2.GPV.GLAST.GPIA,Gross intake ratio to the last grade of lower ...


In [24]:

# sdg_data.head()

In [25]:
sdg_merge=sdg_data.merge(sdg, on='indicator_id', how='left')
sdg_merge.drop(0, inplace=True)
sdg_merge['year']=sdg_merge['year'].apply(pd.to_numeric)
sdg_merge['indicator_label_en']=sdg_merge['indicator_label_en'].astype(str)
sdg_merge.drop_duplicates(subset=['year', 'indicator_label_en'], inplace=True)
sdg_merge.head()


,indicator_id,country_id,year,value,magnitude,qualifier,indicator_label_en
1,EA.4T8.AG25T99.M,SDN,1983,1.333469986915588,NaN,NaN,"Educational attainment rate, completed post-se..."
2,EA.3T8.AG25T99.F,SDN,1983,1.200000047683716,NaN,NaN,"Educational attainment rate, completed upper s..."
3,EA.4T8.AG25T99.F,SDN,1983,.300000011920929,NaN,NaN,"Educational attainment rate, completed post-se..."
4,EA.3T8.AG25T99,SDN,1983,2.799999952316284,NaN,NaN,"Educational attainment rate, completed upper s..."
5,EA.4T8.AG25T99.GPIA,SDN,1983,.2249799966812134,NaN,NaN,"Educational attainment rate, completed post-se..."


In [26]:
# sdg_merge.pivot(columns=['indicator_label_en'], values = 'value', index=['year']).reset_index().head()

In [27]:
demographics= demographics.merge(sdg_merge.pivot(columns=['indicator_label_en'], values = 'value', index=['year']).reset_index(),
                                 left_on='Year',
                                 right_on='year',
                                 how='left'
                                 ).drop(columns='year')

In [44]:
gender = pd.read_csv('./MyDrive/MyDrive/omneda_sudan/gender_sdn.csv')
gender.drop(0, inplace=True)
gender['Indicator Name']=gender['Indicator Name'].astype(str)
gender['Year'] = indicators['Year'].apply(pd.to_numeric)
gender.drop_duplicates(subset=['Year', 'Indicator Name'], inplace=True)
gender.head()

,Country Name,Country ISO3,Year,Indicator Name,Indicator Code,Value
1,Sudan,SDN,2022.0,Firms with female top manager (% of firms),IC.FRM.FEMM.ZS,3.4
2,Sudan,SDN,2021.0,Firms with female participation in ownership (...,IC.FRM.FEMO.ZS,8.2
3,Sudan,SDN,2020.0,"Literacy rate, youth female (% of females ages...",SE.ADT.1524.LT.FE.ZS,73.4899978637695
4,Sudan,SDN,2019.0,"Literacy rate, youth female (% of females ages...",SE.ADT.1524.LT.FE.ZS,100
5,Sudan,SDN,2018.0,"Literacy rate, youth female (% of females ages...",SE.ADT.1524.LT.FE.ZS,64.8099975585938


In [45]:
gender.pivot(columns=['Indicator Name'], values = 'Value', index=['Year']).reset_index().head()

Indicator Name,Year,A woman can apply for a passport in the same way as a man (1=yes; 0=no),"A woman can be ""head of household"" in the same way as a man (1=yes; 0=no)",A woman can choose where to live in the same way as a man (1=yes; 0=no),A woman can get a job in the same way as a man (1=yes; 0=no),A woman can obtain a judgment of divorce in the same way as a man (1=yes; 0=no),A woman can open a bank account in the same way as a man (1=yes; 0=no),A woman can register a business in the same way as a man (1=yes; 0=no),A woman can sign a contract in the same way as a man (1=yes; 0=no),A woman can travel outside her home in the same way as a man (1=yes; 0=no),...,"Vulnerable employment, female (% of female employment) (modeled ILO estimate)","Vulnerable employment, male (% of male employment) (modeled ILO estimate)","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)",Wanted fertility rate (births per woman),Women Business and the Law Index Score (scale 1-100),Women and men have equal ownership rights to immovable property (1=yes; 0=no),Women who were first married by age 15 (% of women ages 20-24),Women who were first married by age 18 (% of women ages 20-24),Women's share of population ages 15+ living with HIV (%)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1960.0,NaN,NaN,0,NaN,NaN,1,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1961.0,NaN,NaN,0,NaN,NaN,1,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,17.5,NaN,NaN,NaN,NaN
3,1962.0,NaN,NaN,0,NaN,NaN,1,1,1,NaN,...,NaN,NaN,NaN,NaN,NaN,17.5,NaN,NaN,NaN,NaN
4,1963.0,NaN,NaN,0,NaN,NaN,1,1,1,NaN,...,65.0328877800454,55.2904139745373,NaN,NaN,NaN,17.5,NaN,NaN,NaN,NaN


In [46]:
demographics= demographics.merge(gender.pivot(columns=['Indicator Name'], values = 'Value', index=['Year']).reset_index(),
                                 left_on='Year',
                                 right_on='Year',
                                 how='left'
                                 )
# .drop(columns='year')

In [47]:
demographics.head()

,Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,...,"Vulnerable employment, female (% of female employment) (modeled ILO estimate)_y","Vulnerable employment, male (% of male employment) (modeled ILO estimate)_y","Wage and salaried workers, female (% of female employment) (modeled ILO estimate)_y","Wage and salaried workers, male (% of male employment) (modeled ILO estimate)_y",Wanted fertility rate (births per woman)_y,Women Business and the Law Index Score (scale 1-100)_y,Women and men have equal ownership rights to immovable property (1=yes; 0=no)_y,Women who were first married by age 15 (% of women ages 20-24)_y,Women who were first married by age 18 (% of women ages 20-24)_y,Women's share of population ages 15+ living with HIV (%)_y
0,5185,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5186,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5187,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5188,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5189,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
demographics.to_csv('demographics.csv')